## 서울 시민 금융 데이터를 활용한 탄소 배출량 및 폐기물 분석 시각화 고도화

> 서울시민들의 카드사용 데이터를 활용하여 소비패턴에 따른 탄소배출량 분석, <br/> 
> 데이터 기반으로 저감 방안에 대한 아이디어 도출  <br/>

### 세부 설명 

1. 업종별 원단위 탄소배출량 추정 
    - (예: 음식점 1,000원당 탄소배출량 500g) 
2. (1) 의 추정값을 카드사용 데이터와 merging
3. 행정동을 분석단위로 하여 탄소배출량을 종속 변수로, 소비패턴(업종별 카드사용내역)을 독립변수로 회귀분석 진행 
4. (3) 의 분석을 진행할 시 행정동별 통제 필요한 변수 추가(예: 주민등록인구, 사업체 수 등) 
5. 또한 필요 시 주요한 개별 업종 등 데이터 subset으로 추가 분석 진행 
6. 행정동별 소비패턴 및 탄소배출량 등 특징/차이 시각화 

### 데이터셋 및 참조 url

1. [서울시민의 업종별 카드소비 패턴 데이터](https://bigdata.seoul.go.kr/data/selectSampleData.do?r_id=P213&sample_data_seq=318&tab_type=A&sch_cate=90&file_id=&sch_text=%EC%8B%A0%ED%95%9C&sch_order=U&currentPage=1)
2. [탄소 발자국 계산 식](https://www.kcen.kr/tanso/intro.green)

In [2]:

%pip install numpy
%pip install pandas
%pip install matplotlib
%pip install sklearn
%pip install seaborn

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [3]:

# 사전 세팅 및 import
# To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all" # "all" 이면, jupyter output 여러개, 한 꺼 번에 볼 수 있음 

import math
import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
import seaborn as sns

# 그래프 한글 깨짐 방지
import matplotlib.font_manager as fm

is_font_changed = False
for target_font in fm.get_font_names():
    if "Malgun Gothic" in target_font:
        # print(target_font)
        plt.rc('font', family='Malgun Gothic')
        is_font_changed = True
        break

if not is_font_changed: 
    plt.rc('font', family='AppleGothic')
    plt.rcParams['axes.unicode_minus'] = False


DATA_PATH: str = "./datas/cep_data"
FILE_LIST = os.listdir(DATA_PATH) # 해당 directory에 file name을 모두 불러와 List로 저장
FILE_LIST

def get_df_from_target_data(file_idx: int):
    try:
        return pd.read_csv(DATA_PATH + "/" + FILE_LIST[file_idx], index_col = False, encoding='utf-8')
    except UnicodeDecodeError:
        return pd.read_csv(DATA_PATH + "/" + FILE_LIST[file_idx], index_col = False, encoding='cp949')


['롯데멤버스_상품분류표.csv',
 '구매지역및구매자_지역코드.csv',
 '블록별 시간대별소액결제건수 카드소비패턴.csv',
 '카드소비 업종코드.csv',
 '판매지기준 행정동별 상품판매.csv',
 '판매지별 구매자지역기준 블록별 상품판매.csv',
 '판매지기준 블록별 상품판매.csv',
 '집계구별 일별소비지역별 카드소비패턴.csv',
 '집계구별 일별시간대별 카드소비패턴.csv',
 '판매지별 구매자지역기준 행정동별 상품판매.csv',
 '블록별 성별연령대별 카드소비패턴.csv']

In [4]:

# TODO: STEP1 
# 카드 소비 업종 코드 별로 원 단위당 총 탄소 배출량 계산 
# -> 소분류 단위로, 업종코드가 다르기 때문에 소분류 대상으로 계산기가 있다고 가설
카드소비_업종코드 = get_df_from_target_data(3)
카드소비_업종코드['업종코드(UPJONG_CD)'] = 카드소비_업종코드['업종코드(UPJONG_CD)'].str.upper()

# 임시 계산식으로 위 데이터프레임에 원 단위 계산값이 들어감
카드소비_업종코드['탄소배출량 단위/원'] = np.random.randint(1, 100, 카드소비_업종코드.shape[0])
카드소비_업종코드

# TODO: STEP2
# 위 계산식 값과 신한_블록별_성별연령대별_카드소비패턴 데이터 프레임 merge
# merge 후 "총 탄소 배출량" 컬럼
신한_블록별_성별연령대별_카드소비패턴 = get_df_from_target_data(-1)
신한_블록별_성별연령대별_카드소비패턴 = pd.merge(신한_블록별_성별연령대별_카드소비패턴, 카드소비_업종코드, how='left', left_on='서울시민업종코드(UPJONG_CD)', right_on='업종코드(UPJONG_CD)')
신한_블록별_성별연령대별_카드소비패턴 = 신한_블록별_성별연령대별_카드소비패턴.drop(['업종코드(UPJONG_CD)'], axis='columns')
신한_블록별_성별연령대별_카드소비패턴['총 탄소배출량'] = 신한_블록별_성별연령대별_카드소비패턴.apply(lambda x: x['탄소배출량 단위/원'] * x["카드이용금액계(AMT_CORR)"], axis=1)
df = 신한_블록별_성별연령대별_카드소비패턴
df



,업종코드(UPJONG_CD),대분류(CLASS1),중분류(CLASS2),소분류(CLASS3),탄소배출량 단위/원
0,SS001,요식/유흥,한식,한식,35
1,SS002,요식/유흥,일식/중식/양식,일식,48
2,SS003,요식/유흥,일식/중식/양식,양식,80
3,SS004,요식/유흥,일식/중식/양식,중식,30
4,SS005,요식/유흥,제과/커피/패스트푸드,제과점,11
...,...,...,...,...,...
70,SS084,스포츠/문화/레저,스포츠/문화/레저,스키,96
71,SS090,교육/학원,학원,입시보습학원,82
72,SS091,교육/학원,학원,외국어학원,47
73,SS092,교육/학원,학원,예체능학원,68


,서울시민업종코드(UPJONG_CD),기준년월(YM),고객주소블록코드(BLOCK_CD),성별(GEDNER),연령대별(AGE),카드이용금액계(AMT_CORR),카드이용건수계(USECT_CORR),대분류(CLASS1),중분류(CLASS2),소분류(CLASS3),탄소배출량 단위/원,총 탄소배출량
0,SS013,201906,14037,F,50대,48288,65,유통,할인점/슈퍼마켓,할인점/슈퍼마켓,16,772608
1,SS048,201608,156830,M,30대,132792,10,가정생활/서비스,각종요금,통신,68,9029856
2,SS016,202009,32925,M,50대,100600,25,유통,편의점,편의점,34,3420400
3,SS001,201805,214245,F,30대,213876,40,요식/유흥,한식,한식,35,7485660
4,SS044,201705,279472,M,30대,155930,5,가정생활/서비스,서비스,생활서비스,28,4366040
...,...,...,...,...,...,...,...,...,...,...,...,...
495,SS012,201709,11449,F,30대,75450,45,유통,백화점,백화점,87,6564150
496,SS044,201707,353037,F,30대,118708,25,가정생활/서비스,서비스,생활서비스,28,3323824
497,SS054,201901,418149,F,20대,1056300,15,의료,병원,종합병원,86,90841800
498,SS021,202009,269015,F,70대이상,25150,5,의류/잡화,의복/의류,의복/의류,97,2439550


In [16]:

# TODO: STEP3
# 목표 데이터 프레임 셋으로 변경
# 기준년월(YM) | 고객주소블록코드(BLOCK_CD) - 행정동 | 성별 F (66%) | 연령대(10대) | 연령대(20대) | 연령대(30대) ... | 의류/잡화, 여행/교통  ... 13개 | 
# 총 탄소배출량 (종속변수, 결과, "덧셈") // => 의류/잡화 800000, 1000 * 계수 + 여행/교통 ... 

droped_df = df.drop(['서울시민업종코드(UPJONG_CD)', '대분류(CLASS1)', '중분류(CLASS2)', '탄소배출량 단위/원', '카드이용금액계(AMT_CORR)'], axis='columns')
purpose_df_columns = ["기준년월(YM)", "고객주소블록코드(BLOCK_CD)", "M", "F", "10대", "20대", "30대", "40대", "50대", "60대", "70대이상"]
# purpose_df_columns = ["기준년월(YM)", "M", "F", "10대", "20대", "30대", "40대", "50대", "60대", "70대이상"]
purpose_df_columns.extend(list(카드소비_업종코드["소분류(CLASS3)"].values))
purpose_df_columns.append("총 탄소배출량")

# 행정동 코드 (가정) 리스트
dong_codes = sorted(list(df["고객주소블록코드(BLOCK_CD)"].unique()))

# 목표 데이터 프레임 셋 만들기
purpose_df = pd.DataFrame(columns=purpose_df_columns)
for target_day in sorted(df["기준년월(YM)"].unique()):

    # 고객 주소 블록 코드를 붙이려면, 다시 for loop 하나 더 필요
    for dong in dong_codes:
        print(target_day, dong, "<< 집계중 ")
        pre_query = (droped_df["기준년월(YM)"] == target_day) & (droped_df["고객주소블록코드(BLOCK_CD)"] == dong)
    
        # 기준년월로 filter된 dataframe 생성
        temp = droped_df.loc[pre_query,:]
        temp_row_cnt = temp.shape[0]

        # 성별 cnt
        male_cnt = temp[(temp["성별(GEDNER)"] == "M")].shape[0]
        female_cnt = temp_row_cnt - male_cnt

        # 나이대 cnt
        ages_cnt = list()
        for age in ["10대", "20대", "30대", "40대", "50대", "60대", "70대이상"]:
            cnt = temp[(temp["연령대별(AGE)"] == age)].shape[0]
            ages_cnt.append(cnt)
        # print("ages_cnt >>", ages_cnt)

        # 분류별 cnt
        # 각 분류 필터 row에 "총 탄소배출량" 값이 있고, 해당 값을 다 더해줘야함
        genre_cnt_output_sum = 0
        genre_cnt = list()
        for genre in list(카드소비_업종코드["소분류(CLASS3)"].values):
            genre_temp_df = temp[(temp["소분류(CLASS3)"] == genre)]
            genre_cnt_output_sum += genre_temp_df.sum()["총 탄소배출량"]
            cnt = temp[(temp["소분류(CLASS3)"] == genre)].shape[0]
            genre_cnt.append(cnt)
        # print("genre_cnt >>", genre_cnt)

        inserted_data = list()
        inserted_data.extend([target_day, dong, male_cnt, female_cnt])
        inserted_data.extend(ages_cnt)
        inserted_data.extend(genre_cnt)
        genre_cnt_output_sum = round(genre_cnt_output_sum / 100_000)  # 단위 : 10만, 반올림
        inserted_data.append(genre_cnt_output_sum)
        purpose_df.loc[len(purpose_df)] = inserted_data

# 분석 목표 데이터프레임 완성
purpose_df


201601 1700 << 집계중 
201601 1795 << 집계중 
201601 5648 << 집계중 
201601 7175 << 집계중 
201601 8321 << 집계중 
201601 8406 << 집계중 
201601 8470 << 집계중 
201601 8705 << 집계중 
201601 8880 << 집계중 
201601 9022 << 집계중 
201601 9137 << 집계중 
201601 9153 << 집계중 
201601 9497 << 집계중 
201601 10062 << 집계중 
201601 10209 << 집계중 
201601 10321 << 집계중 
201601 10431 << 집계중 
201601 10506 << 집계중 
201601 10616 << 집계중 
201601 11220 << 집계중 
201601 11449 << 집계중 
201601 11574 << 집계중 
201601 11679 << 집계중 
201601 12035 << 집계중 
201601 12414 << 집계중 
201601 12706 << 집계중 
201601 12814 << 집계중 
201601 12979 << 집계중 
201601 13219 << 집계중 
201601 13519 << 집계중 
201601 13649 << 집계중 
201601 14014 << 집계중 
201601 14037 << 집계중 
201601 14104 << 집계중 
201601 14484 << 집계중 
201601 14563 << 집계중 
201601 14575 << 집계중 
201601 14641 << 집계중 
201601 14643 << 집계중 
201601 14810 << 집계중 
201601 14963 << 집계중 
201601 15070 << 집계중 
201601 15146 << 집계중 
201601 15308 << 집계중 
201601 15710 << 집계중 
201601 15727 << 집계중 
201601 15859 << 집계중 
201601 15992 << 집계중 
20160

In [6]:

# TODO: STEP4 - 시각화
# 위 "purpose_df" 데이터 프레임 기반으로 분석 시작

# 1차


